In [1]:
import os
import numpy as np
import networkx as nx
from deap import base, creator, tools, algorithms

In [2]:
# Параметры
num_graphs = 1000
num_nodes = 5
pop_size = 30  # Размер популяции
num_generations = 10  # Количество поколений

In [3]:
def get_weight(G,path):
    res=0

    for i in range(-1,path.shape[0]-1):
        res+=G[path[i]][path[i+1]]
    return res

In [4]:
# Директория для хранения файлов
script_dir = os.getcwd()
directory = os.path.join(script_dir, 'data/n=5')
os.makedirs(directory, exist_ok=True)
# Инициализация массива весовых матриц и "хороших" путей
weights_matrices = np.zeros((num_graphs, num_nodes, num_nodes))
good_paths = np.zeros((num_graphs, num_nodes), dtype=int)

In [5]:
def evaluate_path(path, graph):
    weight = sum(graph[path[i]][path[i+1]]['weight'] for i in range(len(path) - 1))
    weight += graph[path[-1]][path[0]]['weight']  # Close the cycle
    return (weight,)  # Return as a tuple


In [6]:
# Генетический алгоритм для поиска пути
def find_good_path(graph):
    # Настройка параметров GA
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    toolbox.register("indices", np.random.permutation, num_nodes)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("evaluate", evaluate_path, graph=graph)
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Создание начальной популяции
    population = toolbox.population(n=pop_size)
    
    # Запуск алгоритма
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, verbose=False)
    
    # Находим лучший путь
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual

In [7]:
def quantize_weight(weight, min_value=1, max_value=10, num_bits=8):
    levels = 2 ** num_bits  # For 8 bits, levels = 256
    quantized_value = int((weight - min_value) / (max_value - min_value) * (levels - 1))
    return quantized_value

TODO:сделать так что бы весь датасет был json файлом 

In [8]:
# Генерация графов и их весовых матриц
for i in range(num_graphs):
    G = nx.complete_graph(num_nodes)
    
    for (u, v) in G.edges():
        weight = np.random.uniform(1, 10)
        quantized_weight = quantize_weight(weight)
        G[u][v]['weight'] = quantized_weight

    
    weights_matrix = nx.to_numpy_array(G, weight='weight', dtype=int)
    weights_matrices[i] = weights_matrix
    
    # Нахождение "хорошего" пути с использованием генетического алгоритма
    best_path = find_good_path(G)
    good_paths[i] = best_path

c:\Users\revya\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\revya\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 

In [13]:
with open(os.path.join(directory, "good_paths.npy"), 'wb') as file:
    np.save(file, good_paths)

In [14]:
with open(os.path.join(directory, "weights.npy"), 'wb') as file:
    np.save(file, weights_matrices)

In [15]:
# Количество перестановок для каждого пути

num_permutations = 5
num_nodes = good_paths.shape[1]  # Количество вершин (длина каждого пути)
noisy_paths = np.zeros(shape=(good_paths.shape[0],num_permutations,num_nodes))

In [16]:
# Генерация зашумленных версий каждого пути
for k in range(good_paths.shape[0]):
    path=good_paths[k]
    weights_of_paths=np.zeros(shape=(good_paths.shape[0],num_permutations))
    for i in range(1, num_permutations + 1):
        # Вычисляем количество перемешанных элементов
        num_swaps = num_permutations
        
        # Создаем зашумленный путь
        noisy_path = path.copy()
        swap_indices = np.random.choice(num_nodes, num_swaps, replace=False)
        #print(f"noisy_path.shape {noisy_path.shape}")
        
        # Выполняем перестановки
        for j in range(0, num_swaps - 1, 2):
            noisy_path[swap_indices[j]], noisy_path[swap_indices[j+1]] = noisy_path[swap_indices[j+1]], noisy_path[swap_indices[j]]
        
        # Добавляем зашумленный путь в список
        noisy_paths[k,i-1,:]=noisy_path
        weights_of_paths[k][i]=get_weight(weights_matrices[k])
    
    

In [17]:
# Преобразование списка в numpy массив
np.save(os.path.join(directory,"noisy_paths.npy"), noisy_paths)

In [18]:
noisy_paths.shape

(1000, 5, 5)

In [19]:
good_paths.shape

(1000, 5)

In [48]:
directory = os.path.join(script_dir, 'data')

if os.path.exists(os.path.join(directory,"weights.npy")):
    with open(os.path.join(directory,"weights.npy"), 'rb') as file:
        weights=np.load(file).astype(np.uint8)
        print(weights.shape)
else:
    print("Файл не найден.")

if os.path.exists(os.path.join(directory,"good_paths.npy")):
    with open(os.path.join(directory,"good_paths.npy"), 'rb') as file:
        good_paths=np.load(file)
        
else:
    print("Файл не найден.")


if os.path.exists(os.path.join(directory,"noisy_paths.npy")):
    with open(os.path.join(directory,"noisy_paths.npy"), 'rb') as file:
        noisy_paths=np.load(file).astype(np.uint8)
        
else:
    print("Файл не найден.")


(1000, 30, 30)


In [49]:
noisy_paths.shape


(1000, 10, 30)

In [50]:
noisy_paths[0]

array([[19, 27,  9,  4,  3, 13, 23, 22, 12, 24,  8, 10, 11, 20, 26, 21,
         5, 16,  2, 18, 25, 28, 15,  6,  1,  0,  7, 29, 14, 17],
       [19, 10,  6, 16, 21, 24,  1, 22, 12, 13,  8, 14,  2, 25, 26,  3,
         5,  4, 11, 18, 20, 28, 15,  9,  0, 23,  7, 29, 27, 17],
       [19, 28,  6,  8, 21, 24, 23,  7, 12, 13, 16, 27, 11, 20, 26,  3,
         5,  4,  2, 18,  9, 10, 15, 25,  0,  1, 22, 29, 14, 17],
       [ 6, 10, 19, 16, 21, 13, 23, 12, 22, 11,  8, 27, 24, 20, 26,  3,
         5,  4, 14, 18,  7, 28, 15,  9,  0,  1, 25, 29,  2, 17],
       [19, 10,  6, 28, 21, 13, 23, 22, 12, 24,  8,  2, 14, 20, 26,  3,
         7,  9, 27, 18, 25, 16, 15,  4,  0,  1,  5, 29, 11, 17],
       [19, 10,  6, 29, 21,  1,  8, 22, 25, 24, 23, 27, 11, 20, 26,  3,
         5,  4,  2, 18, 12,  0, 15,  9, 28, 13,  7, 16, 14, 17],
       [ 3, 10, 11, 16, 21, 13, 14, 22, 12, 24,  8, 15,  6, 28, 26, 19,
         5,  4,  2, 18, 25, 20, 27,  9,  0,  1,  7, 29, 23, 17],
       [19, 10,  6,  1, 21,  9, 23, 11, 1

In [23]:
def get_weight(G,path):
    res=0

    for i in range(-1,path.shape[0]-1):
        res+=G[path[i]][path[i+1]]
    return res

In [54]:
get_weight(weights[0],good_paths[0]),get_weight(weights[0],noisy_paths[0][9])

(54, 85)